# Webscrape Vivino website for ratings

This notebook uses an iterative approach to explore scraping of www.vivino.com for red wine ratings data

In [131]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json
# Apply to all pages
import time
import random
import csv

params = {
    'country_code': '',
    'currency_code': '',
    'grape_filter': '',
    'merchant_id': '',
    'min_rating': '',
    'order_by': 'ratings_average',
    'order': 'desc',
    'page': '2',
    'price_range_max': '',
    'price_range_min': '',
    'wine_type_ids[]': '1',
}

url = 'https://www.vivino.com/api/explore/explore'
userhead={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
r = requests.get(url, params=params, headers=userhead)

In [115]:
json_data = r.json()
winedic = {}
winedic['name'] = []
winedic['region'] = []
winedic['country'] = []
winedic['num_reviews'] = []
winedic['rating'] = []
winedic['acidity'] = []
winedic['fizziness'] = []
winedic['intensity'] = []
winedic['sweetness'] = []
winedic['tannin'] = []
winedic['varietal'] = []
winedic['body'] = []

for wine in json_data['explore_vintage']['matches']:
    winedic['name'].append(wine['vintage']['name'])
    try:
        winedic['region'].append(wine['vintage']['wine']['region']['name'])
        winedic['country'].append(wine['vintage']['wine']['region']['country']['name'])
    except:
        winedic['region'].append('')
        winedic['country'].append('')
    
    try:
        winedic['num_reviews'].append(str(wine['vintage']['statistics']['ratings_count']))
        winedic['rating'].append(str(wine['vintage']['statistics']['ratings_average']))
    except: 
        winedic['num_reviews'].append('')
        winedic['rating'].append('')
    try:
        winedic['acidity'].append(str(wine['vintage']['wine']['taste']['structure']['acidity']))
        winedic['fizziness'].append(str(wine['vintage']['wine']['taste']['structure']['fizziness']))
        winedic['intensity'].append(str(wine['vintage']['wine']['taste']['structure']['intensity']))
        winedic['sweetness'].append(str(wine['vintage']['wine']['taste']['structure']['sweetness']))
        winedic['tannin'].append(str(wine['vintage']['wine']['taste']['structure']['tannin']))
    except:
        winedic['acidity'].append('')
        winedic['fizziness'].append('')
        winedic['intensity'].append('')
        winedic['sweetness'].append('')
        winedic['tannin'].append('')
        
    try:
        winedic['varietal'].append(wine['vintage']['wine']['style']['varietal_name'])
        winedic['body'].append(wine['vintage']['wine']['style']['body_description'])
    except:
        winedic['varietal'].append('')
        winedic['body'].append('')
r.close()

In [142]:
def scrape_single_page(json_data, csvwriter):
    for wine in json_data['explore_vintage']['matches']:
        winedic['name']=wine['vintage']['name']
        try:
            winedic['region']=wine['vintage']['wine']['region']['name']
            winedic['country']=wine['vintage']['wine']['region']['country']['name']
        except:
            winedic['region']=''
            winedic['country']=''

        try:
            winedic['num_reviews']=str(wine['vintage']['statistics']['ratings_count'])
            winedic['rating']=str(wine['vintage']['statistics']['ratings_average'])
        except: 
            winedic['num_reviews']=''
            winedic['rating']=''
        try:
            winedic['acidity']=str(wine['vintage']['wine']['taste']['structure']['acidity'])
            winedic['fizziness']=str(wine['vintage']['wine']['taste']['structure']['fizziness'])
            winedic['intensity']=str(wine['vintage']['wine']['taste']['structure']['intensity'])
            winedic['sweetness']=str(wine['vintage']['wine']['taste']['structure']['sweetness'])
            winedic['tannin']=str(wine['vintage']['wine']['taste']['structure']['tannin'])
        except:
            winedic['acidity']=''
            winedic['fizziness']=''
            winedic['intensity']=''
            winedic['sweetness']=''
            winedic['tannin']=''

        try:
            winedic['varietal']=wine['vintage']['wine']['style']['varietal_name']
            winedic['body']=wine['vintage']['wine']['style']['body_description']
        except:
            winedic['varietal']=''
            winedic['body']=''
        
        csvwriter.writerow(winedic.values())  


# Webscrape LCBO for red wines      
params = {
    'country_code': '',
    'currency_code': '',
    'grape_filter': '',
    'merchant_id': '',
    'min_rating': '',
    'order_by': 'ratings_average',
    'order': 'desc',
    'page': '2',
    'price_range_max': '',
    'price_range_min': '',
    'wine_type_ids[]': '1',
}

url = 'https://www.vivino.com/api/explore/explore'
userhead={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
r = requests.get(url, params=params, headers=userhead)
json_data = r.json()
num_bottles = json_data['explore_vintage']['records_matched']
#num_pages = np.ceil(numbottles/12)
num_pages = 3
r.close()

with open('vivino.csv', 'w', encoding='utf-8', newline='') as csvfile:
    bottle_writer = csv.writer(csvfile)
    bottle_writer.writerow(['name','region','country','num_reviews','rating',\
                            'acidity','fizziness','intensity','sweetness','tannin','varietal','body'])
    for i in range(1, num_pages+1):
        params['page'] = str(i)
        r = requests.get(url, params=params, headers=userhead)
        json_data = r.json()
        scrape_single_page(json_data, bottle_writer)
        r.close()
        # Random sleep to avoid getting banned from the server
        time.sleep(random.randint(1,2))
        print('Finished page: ' + str(i) + ' out of ' + str(num_pages))

ConnectionError: HTTPSConnectionPool(host='www.vivino.com', port=443): Max retries exceeded with url: /api/explore/explore?country_code=&currency_code=&grape_filter=&merchant_id=&min_rating=&order_by=ratings_average&order=desc&page=2&price_range_max=&price_range_min=&wine_type_ids%5B%5D=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B884EA9790>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [73]:
for wine in json_data['explore_vintage']['matches']:
    print('\n'+ wine['vintage']['name'])
    try:
        print('\t' + wine['vintage']['wine']['region']['name'])
        print('\t' + wine['vintage']['wine']['region']['country']['name'])
    except:
        print('Region: N/A')
        print('Country: N/A')
    
    try:
        print('\t' + str(wine['vintage']['statistics']['ratings_count']))
        print('\t' + str(wine['vintage']['statistics']['ratings_average']))
    except: 
        print('Review count: N/A')
        print('Rating: N/A')
    try:
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['acidity']))
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['fizziness']))
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['intensity']))
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['sweetness']))
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['tannin']))
    except:
        print('Acidity: N/A')
        print('Fizziness: N/A')
        print('Intensity: N/A')
        print('Sweetness: N/A')
        print('Tannin: N/A')
        
    try:
        print('\t' + wine['vintage']['wine']['style']['varietal_name'])
        print('\t' + wine['vintage']['wine']['style']['body_description'])
    except:
        print('Varietal: N/A')
        print('Body description: N/A')


Hundred Acre Kayli Morgan Vineyard Cabernet Sauvignon 2014
	Napa Valley
	United States
	154
	4.9
	3.342901
	None
	4.795382
	1.7343491
	3.3750257
	Cabernet Sauvignon
	Very full-bodied

Hundred Acre Kayli Morgan Vineyard Cabernet Sauvignon 2015
	Napa Valley
	United States
	93
	4.9
	3.342901
	None
	4.795382
	1.7343491
	3.3750257
	Cabernet Sauvignon
	Very full-bodied

Frank Family Patriarch 2012
	Rutherford
	United States
	83
	4.9
	3.3242133
	None
	4.867668
	1.3652781
	3.291373
	Cabernet Sauvignon
	Very full-bodied

Alves de Sousa Abandonado 2017
	Douro
	Portugal
	82
	4.9
	3.1363275
	None
	4.6009846
	1.7887058
	3.6249406
	Douro Red
	Very full-bodied

Hundred Acre Few and Far Between Cabernet Sauvignon 2013
	Napa Valley
	United States
	62
	4.9
	3.3757792
	None
	4.6979566
	1.8268845
	3.3687925
	Cabernet Sauvignon
	Very full-bodied

Amici Echion Cabernet Sauvignon 2014
	Oakville
	United States
	61
	4.9
	2.8281817
	None
	4.8525486
	1.5168993
	3.273409
	Cabernet Sauvignon
	Very full-bodied

Si